In [72]:
!pip install -q konlpy tqdm tensorflow

In [73]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import zipfile

In [74]:
# KoNLPy 사용을 위한 JAVA 환경 변수 설정
def set_java_environment():
    # KoNLPy에서 JAVA 실행 환경 필요
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
    return

In [75]:
# zip 파일 압축 해제 함수 (구글 드라이브에 zip파일로 상품,리뷰csv파일 올렸렸음)
def unzip_file(zip_path, extract_path):
    import zipfile
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"압축 해제 완료: {extract_path}")

In [76]:
def load_and_filter_reviews(csv_path):
    import pandas as pd
    import re

    df = pd.read_csv(csv_path)

    # 사용할 주요 컬럼만 선택
    df = df[["바코드", "상품분류코드", "상품명", "리뷰어", "평점", "옵션", "사용자 피부 정보", "본문"]]

    # 상품분류코드가 206, 207인 경우 삭제
    df = df[~df["상품분류코드"].isin([206, 207])].reset_index(drop=True)

    # 상품분류코드가 201인 경우 블러셔 키워드 필터링
    keywords = ["블러쉬", "블러시", "블러셔", "치크", "치크밤", "립앤치크"]
    pattern = "|".join(keywords)

    df = df[~(
        (df["상품분류코드"] == 201) &
        (df["상품명"].str.contains(pattern, case=False, na=False))
    )].reset_index(drop=True)

    print(f"필터링된 리뷰 데이터 수: {len(df)}")
    return df


In [77]:
import ast

# 유효한 피부 타입 목록
valid_skin_types = ['복합성', '건성', '트러블성', '지성', '민감성', '약건성', '중성']

def preprocess_skin_type(df, column_name='사용자 피부 정보'):
    """
    사용자 피부 정보가 문자열 또는 리스트 문자열 형태로 되어 있을 때
    주요 피부 타입만 추출해 새로운 컬럼 '사용자 피부타입'을 추가한다.
    """

    def parse_skin_info(value):
        if pd.isnull(value) or value == '[]':
            return []
        try:
            # 리스트 형태 문자열이면 리스트로 변환
            parsed = ast.literal_eval(value)
            if isinstance(parsed, list):
                return [x.strip() for x in parsed]
        except (ValueError, SyntaxError):
            pass
        # 일반 문자열이라면 ',' 기준으로 분리
        return [x.strip() for x in value.split(',')]

    def extract_main_skin_type(skin_info_list):
        for skin in skin_info_list:
            if skin in valid_skin_types:
                return skin
        return None

    # 파싱 및 주요 피부타입 추출
    df['사용자 피부 정보 리스트'] = df[column_name].apply(parse_skin_info)
    df['사용자 피부타입'] = df['사용자 피부 정보 리스트'].apply(extract_main_skin_type)

    return df


In [78]:
def map_skin_types(df, column_name='사용자 피부타입'):
    """
    사용자 피부타입 컬럼에 표준화 매핑을 적용하고,
    매핑되지 않은 항목은 제거한다.
    """

    # 피부타입 표준 매핑
    mapping = {
        '약건성': '건성',
        '트러블성': '민감성',
        '건성': '건성',
        '민감성': '민감성',
        '복합성': '복합성',
        '지성': '지성',
        '중성': '복합성'
    }

    # 매핑 적용
    df[column_name] = df[column_name].map(mapping)

    # 유효한 피부타입만 필터링
    df = df[df[column_name].notnull()].reset_index(drop=True)

    return df


In [79]:
# 리뷰 본문 전처리 함수 정의
def preprocess_text(text):
    text = str(text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[\u200b\u200c\u200d\ufeff]', '', text)
    text = re.sub(r'[^\w\s가-힣❤️💕✨💖💗💘💓💞💝💜💙💚💛💬💥👍❗️👎☺️😂🤣😭😍😘]', '', text)
    text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
    return text.strip()

In [80]:
# 본문 전처리 컬럼럼을 데이터프레임에 추가
def add_clean_text_column(df):
    df = df[df['본문'].notnull()].reset_index(drop=True)
    df["본문_전처리"] = df["본문"].apply(preprocess_text)
    return df

In [81]:
def create_review_dataframe(df, category):
    """
    원본 DataFrame에서 '상품명', '사용자 피부타입', '본문_전처리' 컬럼만 추출하여
    '상품명', '피부타입', '리뷰내용'으로 구성된 새 DataFrame을 반환하고 저장한다.
    """
    review_df = pd.DataFrame({
        '상품명': df['상품명'],
        '피부타입': df['사용자 피부타입'],
        '리뷰내용': df['본문_전처리']
    })

    save_path = f"{category}_피부타입_리뷰.csv"
    review_df.to_csv(save_path, encoding="utf-8-sig", index=False)
    print(f"저장 완료: {save_path}")

    return review_df


In [82]:
# 제품 정보 병합 및 카테고리 매핑
def merge_product_info(df_reviews, df_product_info):
    category_map = {
        "0113": "스킨/토너", "0114": "에센스/세럼/앰플", "0115": "크림",
        "0116": "로션", "0110": "미스트/오일", "0117": "스킨케어세트", "0118": "스킨케어 디바이스",
        "1001": "클렌징폼/젤", "1004": "오일/밤", "1005": "워터/밀크", "1007": "필링&스크럽",
        "1008": "티슈/패드", "1006": "립&아이리무버", "1009": "클렌징 디바이스",
        "0901": "시트팩", "0904": "패드", "0902": "페이셜팩", "0905": "코팩", "0906": "패치",
        "0206": "립메이크업", "0201": "베이스메이크업", "0207": "아이메이크업",
        "1106": "선크림", "1103": "선스틱", "1104": "선쿠션", "1105": "선스프레이/선패치", "1102": "태닝/애프터선"
    }

    df_merged = df_reviews.merge(
        df_product_info[["상품명", "바코드", "상품분류코드", '비건품목']],
        on="상품명",
        how="left"
    )
    df_merged['품목'] = df_merged['상품분류코드'].astype(str).map(category_map)
    return df_merged


In [83]:
def run_pipeline(category, review_path, product_path):
    print(f"\n====== [{category}] 파이프라인 시작 ======")

    # 1. 리뷰 데이터 로드 및 전처리
    df = load_and_filter_reviews(review_path)
    df = preprocess_skin_type(df)
    df = map_skin_types(df)
    df = add_clean_text_column(df)
    df_new = create_review_dataframe(df, category)

    # 결측치 제거
    df_new = df_new[df_new['피부타입'].notna()].reset_index(drop=True)

    # 2. 제품 정보 불러오기 및 병합
    df_products = pd.read_csv(product_path, dtype={"상품분류코드": str})
    df_final = merge_product_info(df_new, df_products)

    # 3. 저장
    output_path = f"{category}_RAG_데이터.csv"
    df_final.to_csv(output_path, encoding="utf-8-sig", index=False)
    print(f"[{category}] 저장 완료: {output_path}")

In [84]:
# 전체 실행
def run_all_pipelines(review_paths, product_paths):
    for category in review_paths:
        run_pipeline(category, review_paths[category], product_paths[category])

In [85]:
set_java_environment()

# 압축 파일 경로들
zip_paths = {
    '스킨케어': {
        'review': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]스킨케어_바코드_리뷰_전처리.zip',
        'product': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]스킨케어_바코드_상품_전처리.zip',
    },
    '메이크업': {
        'review': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]메이크업_바코드_리뷰_전처리.zip',
        'product': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]메이크업_바코드_상품_전처리.zip',
    },
    '선케어': {
        'review': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]선케어_바코드_리뷰_전처리.zip',
        'product': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]선케어_바코드_상품_전처리.zip',
    },
    '마스크팩': {
        'review': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]마스크팩_바코드_리뷰_전처리.zip',
        'product': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]마스크팩_바코드_상품_전처리.zip',
    },
    '클렌징': {
        'review': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]클렌징_바코드_리뷰_전처리.zip',
        'product': '/content/drive/MyDrive/한경x토스/프로젝트/최종 프로젝트/data/[최종]클렌징_바코드_상품_전처리.zip',
    }
}

# 압축 해제 경로
extract_path = '/content/skin_data'

# zip 해제
for category, paths in zip_paths.items():
    unzip_file(paths['review'], extract_path)
    unzip_file(paths['product'], extract_path)

# 해제된 CSV 경로 정의
review_paths = {
    category: f"{extract_path}/[최종]{category}_바코드_리뷰_전처리.csv"
    for category in zip_paths
}

product_paths = {
    category: f"{extract_path}/[최종]{category}_바코드_상품_전처리.csv"
    for category in zip_paths
}

run_all_pipelines(review_paths, product_paths)

압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data
압축 해제 완료: /content/skin_data

====== [스킨케어] 파이프라인 시작 ======


<ipython-input-76-341517490>:5: DtypeWarning: Columns (6,8,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


필터링된 리뷰 데이터 수: 1119488
저장 완료: 스킨케어_피부타입_리뷰.csv
[스킨케어] 저장 완료: 스킨케어_RAG_데이터.csv

====== [메이크업] 파이프라인 시작 ======


<ipython-input-76-341517490>:5: DtypeWarning: Columns (8,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


필터링된 리뷰 데이터 수: 290319
저장 완료: 메이크업_피부타입_리뷰.csv
[메이크업] 저장 완료: 메이크업_RAG_데이터.csv

====== [선케어] 파이프라인 시작 ======


<ipython-input-76-341517490>:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


필터링된 리뷰 데이터 수: 242922
저장 완료: 선케어_피부타입_리뷰.csv
[선케어] 저장 완료: 선케어_RAG_데이터.csv

====== [마스크팩] 파이프라인 시작 ======


<ipython-input-76-341517490>:5: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


필터링된 리뷰 데이터 수: 618994
저장 완료: 마스크팩_피부타입_리뷰.csv
[마스크팩] 저장 완료: 마스크팩_RAG_데이터.csv

====== [클렌징] 파이프라인 시작 ======
필터링된 리뷰 데이터 수: 525503
저장 완료: 클렌징_피부타입_리뷰.csv
[클렌징] 저장 완료: 클렌징_RAG_데이터.csv
